# 17. Parameter fitting on _A. nidulans_ data

## Numerical experiments - Week 21/2025

_Boyan Mihaylov, MSc Computational Science (UVA/VU)_

Based on the more abundant, time-based data from the _A. niger_ experiments, the most successful models from the previous fitting procedure are used to explain germination data from experiments on _A. nidulans_ conidia [1].

## Prerequisite libraries

In [1]:
using PyPlot
using JLD2
using Statistics
using Distributions
using Revise
using Printf

Revise.includet("./conversions.jl")
Revise.includet("./diffusion.jl")
Revise.includet("./setup.jl")
Revise.includet("./plotting.jl")
Revise.includet("./analysis.jl")
Revise.includet("./datautils.jl")
Revise.includet("./germstats.jl")
using .Conversions
using .Diffusion
using .Setup
using .Plotting
using .Analysis
using .DataUtils
using .GermStats

## 1. Fitting considerations

The _A. nidulans_ dataset does not contain values from which time-dependent evolutions can be reconstructed. Nonetheless, it is noted the measurements are taken at 11 hours of incubation. An alternative study [2] state that the half-saturation time $\tau_g$ for germ tube formation in _A. nidulans_ in a comparable medium is 9.67 hours, the germination rate saturation is 85.46% and the respective design parameter $\nu$ is 6.28. The spore density used in this study is $2.7\times10^5$ spores/mL - at this density, the 11-hour germination rate aligned with the measurements by Herrero-Garcia is estimated to be about 80% [1].

The conflicting findings in the two experiments are rooted in the fact that the 11-hour measurement time in Herrero-Garcia et al [1] is closer to the half-saturation time in the alternative study [2] rather than to a full saturation time. This could suggest that the Dantigny parameters for the data in Herrero-Garcia et al. are different from those in Ijadpanahsaravi et al, likely due to differences in the strain and the cultivation medium used. On the other hand, Herrero-Garcia et al. do not explicitly specify the method used to classify conidia as germinated, which could imply that the percentages also include swollen spores. In any case, it is plausible to assume that the indicated density-dependent germination rates are equilibrium values, i.e. they would not increase significantly under a continued experiment.

With this in consideration, the fitting procedure is required to use **equilibrium probabilities** only. The three models chosen for this are
- inducer-dependent inhibitor threshold and release;
- 2-factor germination with inhibitor-dependent induction threshold;
- independent 2-factor germination with varying permeability.

All models may incorporate a slowly permeating carbon signal, but this assumption becomes irrelevant in the equilibrium limit.

## 2. Equilibrium probabilities

### 2.1. Inducer-dependent inhibitor threshold and release

The time-dependent probability in the first model follows the expression

$$
\begin{equation}
p(\textrm{germination})=\int_0^\infty{\int_0^\infty{\left[1-\Phi{\left(\frac{\beta{(\xi, \pi_{\textrm{eff}})}-\mu_\gamma}{\sigma_\gamma}\right)}\right]\ f_\pi{(\pi_{\textrm{eff}})}\ f_\xi{(\xi)}\ d{\pi_{\textrm{eff}}}}\ d{\xi}}.
\end{equation}
$$

It was previously shown that, in the equilibrium limit, $\beta=\phi$, so the distribution of $\pi_{\textrm{eff}}$, along with its integral, becomes obsolete. Therefore, the new probability takes the form

$$
\begin{equation}
p_{\textrm{eq}}(\textrm{germination})=\int_0^\infty{\left[1-\Phi{\left(\frac{\phi{(\xi)}-\mu_\gamma}{\sigma_\gamma}\right)}\right]\ f_\xi{(\xi)}\ d{\xi}}.
\end{equation}
$$

This is a simple integral which can be solved without resorting to Gauss-Hermite approximation.

If an exogenous concentration of inhibitor is considered upon inoculation, the equilibrium limit of the residual inhibitor concentration in the spore becomes

$$
\begin{equation}
c_{\textrm{in}}^{\textrm{eq}}=\phi \psi+(1-\phi)c_{\textrm{ex}}.
\end{equation}
$$

With the condition for inhibitor depletion,

$$
\begin{equation}
\phi \psi+(1-\phi)c_{\textrm{ex}}<\gamma \psi,
\end{equation}
$$

the germination probability can be formulated as

$$
\begin{equation}
p_{\textrm{eq}}(\textrm{germination})=\int_0^\infty{\int_0^\infty{\left[1-\Phi{\left(\frac{\phi{(\xi)}+(1-\phi)\frac{c_{\textrm{ex}}}{\psi}-\mu_\gamma}{\sigma_\gamma}\right)}\right]\ f_\psi{(\psi)}\ f_\xi{(\xi)}\ d{\psi}}\ d{\xi}}.
\end{equation}
$$

### 2.2. Two-factor germination with inhibitor-dependent induction threshold

The time-dependent probability in the second model follows the expression

$$
\begin{equation}
p(\textrm{germination})=\int_0^\infty{\int_0^\infty{\left[1-\Phi{\left(\frac{\beta{(\xi, t)}-\mu_\gamma}{\sigma_\gamma}\right)}\right]\left[\int_0^\infty{\Phi{\left(\frac{s{(\xi, \kappa, t)}-kc_{\textrm{in}}{(\xi, \psi)}-\mu_{\omega_0}}{\sigma_{\omega_0}}\right)}\ f_\psi{(\psi)}\ d{\psi}}\right] f_\kappa{(\kappa)}\ f_\xi{(\xi)}\ d{\kappa}}\ d{\xi}}.
\end{equation}
$$

It was previously shown that, in the equilibrium limit, $\beta=\phi$, hence the equilibrium value of $c_{\textrm{in}}^{\textrm{eq}}=\phi\psi$. Since $c_{\textrm{in}}\to 0$ and $\phi$ is only dependent on $\xi$, the integral over $\psi$ can be omitted. In the long-time limit, the carbon source concentration at the receptor proteins equalises to the ambient concentration in the medium, $c_{\textrm{out}}^{\textrm{cs}}$, so the integral over $\kappa$ can be discarded. Thus, the equilibrium signal strength becomes

$$
\begin{equation}
s_{\textrm{eq}}=\frac{c_{\textrm{out}}^{\textrm{cs}}}{c_{\textrm{out}}^{\textrm{cs}}+K_{\textrm{cs}}}.
\end{equation}
$$

Taking this together, the germination probability simplifies to

$$
\begin{equation}
p_{\textrm{eq}}(\textrm{germination})=\int_0^\infty{\left[1-\Phi{\left(\frac{\phi{(\xi)}-\mu_\gamma}{\sigma_\gamma}\right)}\right]\left[\int_0^\infty{\Phi{\left(\frac{s_{\textrm{eq}}-k\phi{(\xi)}\psi-\mu_{\omega_0}}{\sigma_{\omega_0}}\right)}\ f_\psi{(\psi)}\ d{\psi}}\right]f_\xi{(\xi)}\ d{\xi}}.
\end{equation}
$$

In summary, the residual inhibition still affects both the inhibition-determined germination trigger and the threshold of the inducing signal.

Considering the exogenous inhibitor, the probability is reformulated to

$$
\begin{equation}
p_{\textrm{eq}}(\textrm{germination})=\int_0^\infty{\int_0^\infty{\left[1-\Phi{\left(\frac{\phi{(\xi)}+(1-\phi)\frac{c_{\textrm{ex}}}{\psi}-\mu_\gamma}{\sigma_\gamma}\right)}\right]\left[\Phi{\left(\frac{s_{\textrm{eq}}-k\left[\phi{(\xi)}\psi+(1-\phi)c_{\textrm{ex}}\right]-\mu_{\omega_0}}{\sigma_{\omega_0}}\right)}\ f_\psi{(\psi)}\right]f_\xi{(\xi)}\ d{\psi}}\ d{\xi}}.
\end{equation}
$$

### 2.3. Independent two-factor germination with with varying permeability

The time-dependent probability in the third model follows the expression

$$
\begin{equation}
p(\textrm{germination})=\int_0^\infty{\int_0^\infty{\int_0^\infty{\Phi{\left(\frac{s{(\xi, \kappa, t)}-\mu_\omega}{\sigma_\omega}\right)}\left[1-\Phi{\left(\frac{\beta{(\xi, \alpha, t)}-\mu_\gamma}{\sigma_\gamma}\right)}\right]\ f_\alpha{(\alpha)}\ f_\kappa{(\kappa)}\ f_\xi{(\xi)\ d{\alpha}}}\ d{\kappa}}\ d{\xi}}.
\end{equation}
$$

Since $\beta=\phi$ and $\phi$ is only dependent on $\xi$, the integral over $\alpha$ can be omitted. Similarly, in the long-time limit, the carbon source concentration at the receptor proteins equalises to the ambient concentration in the medium, $c_{\textrm{out}}^{\textrm{cs}}$, making the computations with $t$ and $\kappa$ obsolete. Thus, the germination probability simplifies to

$$
\begin{equation}
p_{\textrm{eq}}(\textrm{germination})=\Phi{\left(\frac{s_{\textrm{eq}}-\mu_\omega}{\sigma_\omega}\right)}\int_0^\infty{\left[1-\Phi{\left(\frac{\phi{(\xi)}-\mu_\gamma}{\sigma_\gamma}\right)}\right]\ f_\xi{(\xi)}\ d{\xi}}.
\end{equation}
$$

It should be noted that this probability does not involve any information about the distribution of the permeation constant or the cell wall porosity, so it is identical with the case of a deterministic permeation constant. Since this is a simple integral, it does not require a Gauss-Hermite approximation.

Taking into account the exogenous inhibitor concentration, the germination probability becomes

$$
\begin{equation}
p_{\textrm{eq}}(\textrm{germination})=\Phi{\left(\frac{s_{\textrm{eq}}-\mu_\omega}{\sigma_\omega}\right)}\int_0^\infty{\int_0^\infty{\left[1-\Phi{\left(\frac{\phi{(\xi)}+(1-\phi)\frac{c_{\textrm{ex}}}{\psi}-\mu_\gamma}{\sigma_\gamma}\right)}\right]\ f_\psi{(\psi)}\ f_\xi{(\xi)}\ d{\psi}}\ d{\xi}}.
\end{equation}
$$

## 3. Data and default parameters

The germination data and the corresponding densities are stored in variables below. The parameter boundaries remain as in the previous parameter fitting procedure. The distribution parameters of the spore radii are set to $\mu_\xi=1.275\ \mu\textrm{m}$ and $\sigma_\xi=0.075\ \mu\textrm{m}$ [3].

The external concentration of the carbon source can be computed by considering that the study uses a minimal medium (MMA) as described in Pontecorvo et al. [4]. The carbon source used in this medium is dextrose, of which $10\ \textrm{g}$ are dilluted in 1 liter of water. Having a molar mass of $180\ \textrm{g/mol}$, the dextrose has a concentration of $10\ \textrm{g}/(180\ \textrm{g/mol}\times 1\ \textrm{L})\approx0.0556\ \textrm{M}$.

It is known that _A. nidulans_ germinates up to 50% even in the absence of a carbon source [2]. Capturing this in a negative induction threshold, it implies that $p{(\omega)<0}=0$. This is always true for a Gaussian distribution, so there is effectively no upper boundary for the coefficient of variation of $\omega$. On the other hand, a control experiment without forced inhibition exhibits 95% germination success [1], so the 5% negative threshold assumption from the previous fitting procedure is maintained.

In [3]:
# Germination data
densities = [1e3, 1e4, 1e5, 1e6, 1e7, 1e8]
germ_responses = [0.913, 0.925, 0.83, 0.72, 0.17, 0.08]
conf_intervals = [0.071, 0.074, 0.07, 0.061, 0.003, 0.003]

density_ex = 1e5
ex_concentrations = [1.0, 2.5, 4.0, 5.0] .* 0.001
germ_responses_ex = [0.87, 0.36, 0.14, 0.01]
conf_intervals_ex = [0.061, 0.043, 0.021, 0.003]

# Define default parameters globally
def_params = Dict(
    :μ_ξ => 1.275,
    :σ_ξ => 0.075,
    :c₀_cs => 0.00556
)

# Define parameter bounds globally
bounds_narrow = Dict(
    :k => (-6, 6),
    :K_cs => (1e-8, 0.1),
    :μ_γ => (1e-6, 1e-4),
    :δ_γ => (-10, 10),
    :μ_ω => (1e-6, 10.0),
    :δ_ω => (-10, 0),
    :μ_ψ => (1e-5, 1.0),	
    :δ_ψ => (-10, 0)
)

bounds_broad = Dict(
    :k => (-6, 6),
    :K_cs => (1e-8, 10.0),
    :μ_γ => (1e-8, 1e-1),
    :δ_γ => (-10, 10),
    :μ_ω => (1e-6, 10.0),
    :δ_ω => (-10, 0),
    :μ_ψ => (1e-5, 1.0),	
    :δ_ψ => (-10, 0)
)

# Radius distribution
μ_ξ = def_params[:μ_ξ]
σ_ξ = def_params[:σ_ξ]
μ_ξ_log = log(μ_ξ^2 / sqrt(σ_ξ^2 + μ_ξ^2))
σ_ξ_log = sqrt(log(σ_ξ^2 / μ_ξ^2 + 1))
dist_ξ = LogNormal(μ_ξ_log, σ_ξ_log)

LogNormal{Float64}(μ=0.2412190611763456, σ=0.05877273915760268)

## 4. Fitting to density-dependent data

### 4.1. Inducer-dependent inhibitor threshold and release

In [ ]:
max_steps = 10000
params_opt, rmse = fit_model_to_data_equilibrium("inhibitor", def_params, germ_responses, densities, bounds_narrow, max_steps=max_steps)

jldopen("Data/fit2_inhibitor.jld2", "w") do file
    file["def_params"] = def_params
    file["bounds"] = bounds_narrow
    file["params_opt"] = params_opt
    file["max_steps"] = max_steps
    file["rmse"] = rmse
end

params_opt

In [ ]:
matplotlib.pyplot.close()

# Load the fitted parameters
def_params_opt, bounds_opt, params_opt, max_steps_opt, rmse = jldopen("Data/fit2_inhibitor.jld2", "r") do file
    return file["def_params"], file["bounds"], file["params_opt"], file["max_steps"], file["rmse"]
end

# Load default and fitted parameters
params_test = merge(def_params, params_opt)
for (key, value) in params_test
    params_test[key] = value[1]
end

# Reshape inputs
errors = zeros(1, length(germ_responses), 2)
errors[1, :, 1] .= conf_intervals
germ_resp_vis = reshape(germ_responses, 1, length(germ_responses)).*100

# Generate test data
density_exp_range = LinRange(3, 8, 1000)
model_densities = 10 .^ density_exp_range
model_responses = zeros(1, length(model_densities))

# Plot the fitted model against the germination data
fig, ax = subplots(figsize=(7, 5))
model_responses[1, :] .= germ_response_inducer_dep_inhibitor_combined_eq.(inverse_mL_to_cubic_um.(model_densities), dist_ξ, params_test[:μ_γ], params_test[:σ_γ]).*100
plot_germination_data_fit(densities, germ_resp_vis, model_densities, model_responses, ["A. nidulans germination"], yerr=errors.*100, ax=ax)
fig.suptitle("Inducer-dependent inhibitor threshold and release, RMSE: $(round(rmse, sigdigits=3))")

tight_layout()
gcf()

### 4.2. Two-factor germination with inhibitor-dependent induction threshold

In [ ]:
max_steps = 10000
params_opt, rmse = fit_model_to_data_equilibrium("combined_inducer", def_params, germ_responses, densities, bounds_narrow, max_steps=max_steps)

jldopen("Data/fit2_combined_inducer.jld2", "w") do file
    file["def_params"] = def_params
    file["bounds"] = bounds_narrow
    file["params_opt"] = params_opt
    file["max_steps"] = max_steps
    file["rmse"] = rmse
end

params_opt

In [ ]:
matplotlib.pyplot.close()

# Load the fitted parameters
def_params_opt, bounds_opt, params_opt, max_steps_opt, rmse = jldopen("Data/fit2_combined_inducer.jld2", "r") do file
    return file["def_params"], file["bounds"], file["params_opt"], file["max_steps"], file["rmse"]
end

println(rmse)

# Load default and fitted parameters
params_test = merge(def_params, params_opt)
for (key, value) in params_test
    params_test[key] = value[1]
end

# Reshape inputs
errors = zeros(1, length(germ_responses), 2)
errors[1, :, 1] .= conf_intervals
germ_resp_vis = reshape(germ_responses, 1, length(germ_responses)).*100

# Generate test data
density_exp_range = LinRange(3, 8, 1000)
model_densities = 10 .^ density_exp_range
model_responses = zeros(1, length(model_densities))

# Plot the fitted model against the germination data
fig, ax = subplots(figsize=(7, 5))
model_responses[1, :] .= germ_response_inhibitor_dep_inducer_thresh_2_factors_eq.(inverse_mL_to_cubic_um.(model_densities), dist_ξ, def_params[:c₀_cs],
                                                                                params_test[:K_cs], params_test[:k],
                                                                                params_test[:μ_γ], params_test[:σ_γ],
                                                                                params_test[:μ_ω], params_test[:σ_ω],
                                                                                params_test[:μ_ψ], params_test[:σ_ψ]).*100
plot_germination_data_fit(densities, germ_resp_vis, model_densities, model_responses, ["A. nidulans germination"], yerr=errors.*100, ax=ax)
fig.suptitle("Inducer-dependent inhibitor threshold and release, RMSE: $(round(rmse, sigdigits=3))")

tight_layout()
gcf()

### 4.3. Independent two-factor germination

In [ ]:
max_steps = 10000
params_opt, rmse = fit_model_to_data_equilibrium("independent", def_params, germ_responses, densities, bounds_broad, max_steps=max_steps)

jldopen("Data/fit2_independent.jld2", "w") do file
    file["def_params"] = def_params
    file["bounds"] = bounds_broad
    file["params_opt"] = params_opt
    file["max_steps"] = max_steps
    file["rmse"] = rmse
end

params_opt

In [ ]:
matplotlib.pyplot.close()

# Load the fitted parameters
def_params_opt, bounds_opt, params_opt, max_steps_opt, rmse = jldopen("Data/fit2_independent.jld2", "r") do file
    return file["def_params"], file["bounds"], file["params_opt"], file["max_steps"], file["rmse"]
end

println(rmse)

# Load default and fitted parameters
params_test = merge(def_params, params_opt)
for (key, value) in params_test
    params_test[key] = value[1]
end

# Reshape inputs
errors = zeros(1, length(germ_responses), 2)
errors[1, :, 1] .= conf_intervals
germ_resp_vis = reshape(germ_responses, 1, length(germ_responses)).*100

# Generate test data
density_exp_range = LinRange(2, 9, 1000)
model_densities = 10 .^ density_exp_range
model_responses = zeros(1, length(model_densities))

# Plot the fitted model against the germination data
fig, ax = subplots(figsize=(7, 5))
model_responses[1, :] .= germ_response_independent_eq.(inverse_mL_to_cubic_um.(model_densities), dist_ξ,
                                                        params_test[:c₀_cs], params_test[:K_cs],
                                                        params_test[:μ_γ], params_test[:σ_γ],
                                                        params_test[:μ_ω], params_test[:σ_ω]).*100
plot_germination_data_fit(densities, germ_resp_vis, model_densities, model_responses, ["A. nidulans germination"], yerr=errors.*100, ax=ax)
fig.suptitle("Independent inducer and inhibitor, RMSE: $(round(rmse, sigdigits=3))")

tight_layout()
gcf()

## 5. Fitting to exogenously driven inhibition data

### 5.1. Inducer-dependent inhibitor threshold and release

In [ ]:
max_steps = 10000
params_opt, rmse = fit_model_to_data_equilibrium("inhibitor_ex", def_params, germ_responses_ex, inverse_mL_to_cubic_um(density_ex), bounds_narrow, c_ex_vals=ex_concentrations, max_steps=max_steps)

jldopen("Data/fit2_inhibitor_ex.jld2", "w") do file
    file["def_params"] = def_params
    file["bounds"] = bounds_narrow
    file["params_opt"] = params_opt
    file["max_steps"] = max_steps
    file["rmse"] = rmse
end

params_opt

In [ ]:
matplotlib.pyplot.close()

# Load the fitted parameters
def_params_opt, bounds_opt, params_opt, max_steps_opt, rmse = jldopen("Data/fit2_inhibitor_ex.jld2", "r") do file
    return file["def_params"], file["bounds"], file["params_opt"], file["max_steps"], file["rmse"]
end

println(rmse)

# Load default and fitted parameters
params_test = merge(def_params, params_opt)
for (key, value) in params_test
    params_test[key] = value[1]
end

# Reshape inputs
errors = zeros(1, length(germ_responses_ex), 2)
errors[1, :, 1] .= conf_intervals_ex
germ_resp_vis = reshape(germ_responses_ex, 1, length(germ_responses_ex)).*100

# Generate test data
model_concentrations = LinRange(0.0, ex_concentrations[end], 1000)
model_responses = zeros(1, length(model_concentrations))

# Plot the fitted model against the germination data
fig, ax = subplots(figsize=(7, 5))
model_responses[1, :] .= germ_response_inducer_dep_inhibitor_combined_eq_c_ex.(inverse_mL_to_cubic_um(density_ex), dist_ξ, model_concentrations,
                                                                                params_test[:μ_γ], params_test[:σ_γ], params_test[:μ_ψ], params_test[:σ_ψ]).*100
plot_germination_data_fit(ex_concentrations, germ_resp_vis, model_concentrations, model_responses, ["A. nidulans germination"], yerr=errors.*100, ax=ax, c_ex=true)
fig.suptitle("Independent inducer and inhibitor, RMSE: $(round(rmse, sigdigits=3))")

tight_layout()
gcf()

### 5.2. Two-factor germination with inhibitor-dependent induction threshold

In [ ]:
max_steps = 10000
params_opt, rmse = fit_model_to_data_equilibrium("combined_inducer_ex", def_params, germ_responses_ex, inverse_mL_to_cubic_um(density_ex), bounds_narrow, c_ex_vals=ex_concentrations, max_steps=max_steps)

jldopen("Data/fit2_combined_inducer_ex.jld2", "w") do file
    file["def_params"] = def_params
    file["bounds"] = bounds_narrow
    file["params_opt"] = params_opt
    file["max_steps"] = max_steps
    file["rmse"] = rmse
end

params_opt

In [ ]:
matplotlib.pyplot.close()

# Load the fitted parameters
def_params_opt, bounds_opt, params_opt, max_steps_opt, rmse = jldopen("Data/fit2_combined_inducer_ex.jld2", "r") do file
    return file["def_params"], file["bounds"], file["params_opt"], file["max_steps"], file["rmse"]
end

println(rmse)

# Load default and fitted parameters
params_test = merge(def_params, params_opt)
for (key, value) in params_test
    params_test[key] = value[1]
end

# Reshape inputs
errors = zeros(1, length(germ_responses_ex), 2)
errors[1, :, 1] .= conf_intervals_ex
germ_resp_vis = reshape(germ_responses_ex, 1, length(germ_responses_ex)).*100

# Generate test data
model_concentrations = LinRange(0.0, ex_concentrations[end], 1000)
model_responses = zeros(1, length(model_concentrations))

# Plot the fitted model against the germination data
fig, ax = subplots(figsize=(7, 5))
model_responses[1, :] .= germ_response_inhibitor_dep_inducer_thresh_2_factors_eq_c_ex.(inverse_mL_to_cubic_um(density_ex), dist_ξ, model_concentrations,
                                                                                        params_test[:c₀_cs], params_test[:K_cs], params_test[:k],
                                                                                        params_test[:μ_γ], params_test[:σ_γ],
                                                                                        params_test[:μ_ω], params_test[:σ_ω],
                                                                                        params_test[:μ_ψ], params_test[:σ_ψ]).*100
plot_germination_data_fit(ex_concentrations, germ_resp_vis, model_concentrations, model_responses, ["A. nidulans germination"], yerr=errors.*100, ax=ax, c_ex=true)
fig.suptitle("Independent inducer and inhibitor, RMSE: $(round(rmse, sigdigits=3))")

tight_layout()
gcf()

### 5.3. Independent two-factor germination

In [ ]:
max_steps = 10000
params_opt, rmse = fit_model_to_data_equilibrium("independent_ex", def_params, germ_responses_ex, inverse_mL_to_cubic_um(density_ex), bounds_narrow, c_ex_vals=ex_concentrations, max_steps=max_steps)

jldopen("Data/fit2_independent_ex.jld2", "w") do file
    file["def_params"] = def_params
    file["bounds"] = bounds_narrow
    file["params_opt"] = params_opt
    file["max_steps"] = max_steps
    file["rmse"] = rmse
end

params_opt

In [ ]:
matplotlib.pyplot.close()

# Load the fitted parameters
def_params_opt, bounds_opt, params_opt, max_steps_opt, rmse = jldopen("Data/fit2_independent_ex.jld2", "r") do file
    return file["def_params"], file["bounds"], file["params_opt"], file["max_steps"], file["rmse"]
end

println(rmse)

# Load default and fitted parameters
params_test = merge(def_params, params_opt)
for (key, value) in params_test
    params_test[key] = value[1]
end

# Reshape inputs
errors = zeros(1, length(germ_responses_ex), 2)
errors[1, :, 1] .= conf_intervals_ex
germ_resp_vis = reshape(germ_responses_ex, 1, length(germ_responses_ex)).*100

# Generate test data
model_concentrations = LinRange(0.0, ex_concentrations[end], 1000)
model_responses = zeros(1, length(model_concentrations))

# Plot the fitted model against the germination data
fig, ax = subplots(figsize=(7, 5))
model_responses[1, :] .= germ_response_independent_eq_c_ex.(inverse_mL_to_cubic_um(density_ex), dist_ξ, model_concentrations,
                                                            params_test[:c₀_cs], params_test[:K_cs],
                                                            params_test[:μ_γ], params_test[:σ_γ],
                                                            params_test[:μ_ω], params_test[:σ_ω],
                                                            params_test[:μ_ψ], params_test[:σ_ψ]).*100
plot_germination_data_fit(ex_concentrations, germ_resp_vis, model_concentrations, model_responses, ["A. nidulans germination"], yerr=errors.*100, ax=ax, c_ex=true)
fig.suptitle("Independent inducer and inhibitor, RMSE: $(round(rmse, sigdigits=3))")

tight_layout()
gcf()

## References

- [1] Herrero-García, E., Garzia, A., Cordobés, S., Espeso, E.A., & Ugalde, U. (2011). 8-Carbon oxylipins inhibit germination and growth, and stimulate aerial conidiation in Aspergillus nidulans. Fungal biology, 115 4-5, 393-400.
- [2] Ijadpanahsaravi, M., Teertstra, W.R., & Wösten, H.A. (2022). Inter- and intra-species heterogeneity in germination of Aspergillus conidia. Antonie Van Leeuwenhoek, 115, 1151 - 1164.
- [3] Yu, J. (2010). Regulation of Development in Aspergillus nidulans and Aspergillus fumigatus. Mycobiology, 38, 229 - 237.
- [4] Pontecorvo, G., Roper, J.A., Chemmons, L.M., Macdonald, K.D., & Bufton, A. (1953). The genetics of Aspergillus nidulans. Advances in genetics, 5, 141-238 .